# Building the simplest Graph

We start with a graph with two nodes connected by one edge. 


In [1]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 409.5 kB/s eta 0:00:000:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 1.7 MB/s eta 0:00:00 MB/s eta 0:00:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.27
    Uninstalling langchain-core-0.1.27:
      Successfully uninstalled langchain-core-0.1.27
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.0.8 requires langchain-core<0.2.0,>=0.1.27, but you have langchain-core 0.2.0 which is incompatible.
langchain-community 0.0.24 requires langchain-core<0.2,>=0.1.26, but you have langchain-core 0.2.0 which is incompatible.
langchain 0.1.9 requires langchain-core<0.2,>=0.1.26, but you have langchain-core 0.2.0 which is incompatible.


Nodes act like functions that can be called as needed. In our case Node 1 is our starting point and Node 2 is our finish point.

In [2]:
def function_1(input_1):
    return input_1 + " Hi "

def function_2(input_2):
    return input_2 + "there"

In [3]:
from langgraph.graph import Graph

# Define a Langchain graph
workflow = Graph()

workflow.add_node("node_1", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge('node_1', 'node_2')

workflow.set_entry_point("node_1")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [4]:
app.invoke("Hello")

'Hello Hi there'

In [5]:
input = 'Hello'
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'node_1':
---
Hello Hi 

---

Output from node 'node_2':
---
Hello Hi there

---



### As you can see, we can run the nodes as functions and return some values from them. 



# Adding LLM Call

Now, let's make the first node as an "Agent" that can call Open AI models. We can use langchain to make this call easy for us. 

In [6]:
!pip install langchain langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 4.9 MB/s eta 0:00:003.1 MB/s eta 0:00:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.0
    Uninstalling langchain-core-0.2.0:
      Successfully uninstalled langchain-core-0.2.0


A usual call to ChatOpenAI model in LangChain is done as below:

First set your API keys for OpenAI

In [7]:
!pip install python-dotenv

In [42]:
from dotenv import load_dotenv, find_dotenv
import os

# Find and load the .env file
dotenv_path = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)
    print(f".env file loaded from: {dotenv_path}")
else:
    print("No .env file found")

from dotenv import load_dotenv


# Now you can access your environment variables using os.environ
#os.environ['OPENAI_API_KEY'] = os.environ.get("OPENAI_API_KEY")

.env file loaded from: /Users/admin/Documents/Code/langchain-udemy/ice_breaker/.env


In [43]:
'OPENAI_API_KEY' in os.environ

True

In [44]:
from langchain_openai import ChatOpenAI

# Set the model as ChatOpenAI
model = ChatOpenAI(temperature=0) 

#Call the model with a user message
model.invoke('Hey there')

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-39458b25-e4bf-46c4-afea-743ef49ee10a-0')

And if you just want to see the AI response, you can do the following:

In [45]:
model.invoke('Hey there').content

'Hello! How can I assist you today?'

Cool! Keeping that in mind, let's change the function 1 above so that we can send the user question to the model. Then we will send this response to function 2, which will add a short string and return to the user.

In [46]:
def function_1(input_1):
    response = model.invoke(input_1)
    return response.content

def function_2(input_2):
    return "Agent Says: " + input_2

In [47]:
# Define a Langchain graph
workflow = Graph()

#calling node 1 as agent
workflow.add_node("agent", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge('agent', 'node_2')

workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [48]:
app.invoke("Hey there")

'Agent Says: Hello! How can I assist you today?'

In [49]:
input = 'Hey there'
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
Hello! How can I assist you today?

---

Output from node 'node_2':
---
Agent Says: Hello! How can I assist you today?

---



# First functional Agent App - City Temperature

### Step 1: Parse the city mentioned
Let's extract the city that a user mentions in a query

In [50]:
def function_1(input_1):
    complete_query = "Your task is to provide only the city name based on the user query. \
        Nothing more, just the city name mentioned. Following is the user query: " + input_1
    response = model.invoke(complete_query)
    return response.content

def function_2(input_2):
    return "Agent Says: " + input_2

In [51]:
# Define a Langchain graph
workflow = Graph()

#calling node 1 as agent
workflow.add_node("agent", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge('agent', 'node_2')

workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [52]:
app.invoke("What's the temperature in Las Vegas")

'Agent Says: Las Vegas'

### Step 2: Adding a weather API call

What if we want the function 2 to take the city name and give us the weather for that city.

Well we know that Open Weather Map is [integrated](https://python.langchain.com/docs/integrations/tools/openweathermap) into LangChain

We need to install pyown, create an API key on the website of Open Weather Map (which takes a few hours to activate) and then run the cells below to get weather of a given city.

In [53]:
!pip install pyowm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 17.0 MB/s eta 0:00:0031m19.2 MB/s eta 0:00:01


In [54]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper
load_dotenv()
os.environ["OPENWEATHERMAP_API_KEY"] = os.environ.get("OPENWEATHERMAP_API_KEY")

weather = OpenWeatherMapAPIWrapper()

In [55]:
weather_data = weather.run("Las Vegas")
print(weather_data)

In Las Vegas, the current weather is as follows:
Detailed status: clear sky
Wind speed: 8.23 m/s, direction: 200°
Humidity: 18%
Temperature: 
  - Current: 28.78°C
  - High: 30.06°C
  - Low: 26.88°C
  - Feels like: 27.25°C
Rain: {}
Heat index: None
Cloud cover: 0%


Now, let's integrate this into function 2 and call the function two as a "tool" or "weather_agent" instead of "node_2" in our workflow.

In [56]:
def function_1(input_1):
    complete_query = "Your task is to provide only the city name based on the user query. \
        Nothing more, just the city name mentioned. Following is the user query: " + input_1
    response = model.invoke(complete_query)
    return response.content

def function_2(input_2):
    weather_data = weather.run(input_2)
    return weather_data

In [57]:
from langgraph.graph import Graph

workflow = Graph()

#calling node 1 as agent
workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

workflow.add_edge('agent', 'tool')

workflow.set_entry_point("agent")
workflow.set_finish_point("tool")

app = workflow.compile()

In [58]:
app.invoke("What's the temperature in Oakland")

'In Oakland, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 5.66 m/s, direction: 260°\nHumidity: 55%\nTemperature: \n  - Current: 29.38°C\n  - High: 31.1°C\n  - Low: 26.45°C\n  - Feels like: 30.88°C\nRain: {}\nHeat index: None\nCloud cover: 75%'

In [61]:
input = "What's the temperature in oakland"
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
Oakland

---

Output from node 'tool':
---
In Oakland, the current weather is as follows:
Detailed status: broken clouds
Wind speed: 5.66 m/s, direction: 260°
Humidity: 55%
Temperature: 
  - Current: 29.38°C
  - High: 31.1°C
  - Low: 26.45°C
  - Feels like: 30.88°C
Rain: {}
Heat index: None
Cloud cover: 75%

---



### Step 3 Adding another LLM Call to filter results

What if we only want the temperature? But current setup gives us the full weather report. 

Well we can make another LLM call to filter data

In [62]:
def function_3(input_3):
    complete_query = "Your task is to provide info concisely based on the user query. Following is the user query: " + "user input"
    response = model.invoke(complete_query)
    return response.content

But the issue is the user input is not available from node 2.

Can we pass user input all along from first node to the last?

Yes, we can use a dictionary and pass it between nodes (we could also use just a list, but dict makes it a bit easier)

In [63]:
# assign AgentState as an empty dict
AgentState = {}

# messages key will be assigned as an empty array. We will append new messages as we pass along nodes. 
AgentState["messages"] = []

In [64]:
AgentState

{'messages': []}

Our goal is to have this state filled as:
{'messages': [HumanMessage, AIMessage, ...]]}

Also now we need to modify our functions to pass info along the new AgentState

In [65]:
def function_1(state):
    messages = state['messages']
    user_input = messages[-1]
    complete_query = "Your task is to provide only the city name based on the user query. \
                    Nothing more, just the city name mentioned. Following is the user query: " + user_input
    response = model.invoke(complete_query)
    state['messages'].append(response.content) # appending AIMessage response to the AgentState
    return state

def function_2(state):
    messages = state['messages']
    agent_response = messages[-1]
    weather = OpenWeatherMapAPIWrapper()
    weather_data = weather.run(agent_response)
    state['messages'].append(weather_data)
    return state

def function_3(state):
    messages = state['messages']
    user_input = messages[0]
    available_info = messages[-1]
    agent2_query = "Your task is to provide info concisely based on the user query and the available information from the internet. \
                        Following is the user query: " + user_input + " Available information: " + available_info
    response = model.invoke(agent2_query)
    return response.content


In [66]:
from langgraph.graph import Graph

workflow = Graph()


workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)
workflow.add_node("responder", function_3)

workflow.add_edge('agent', 'tool')
workflow.add_edge('tool', 'responder')

workflow.set_entry_point("agent")
workflow.set_finish_point("responder")

app = workflow.compile()

In [67]:
inputs = {"messages": ["what is the temperature in oakland"]}
app.invoke(inputs)

'The current temperature in Oakland is 29.38°C with broken clouds and a humidity of 55%.'

In [68]:
input = {"messages": ["what is the temperature in oakland"]}
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': ['what is the temperature in oakland', 'Oakland']}

---

Output from node 'tool':
---
{'messages': ['what is the temperature in oakland', 'Oakland', 'In Oakland, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 5.66 m/s, direction: 260°\nHumidity: 55%\nTemperature: \n  - Current: 29.38°C\n  - High: 31.1°C\n  - Low: 26.45°C\n  - Feels like: 30.88°C\nRain: {}\nHeat index: None\nCloud cover: 75%']}

---

Output from node 'responder':
---
The current temperature in Oakland is 29.38°C with broken clouds and a humidity of 55%.

---



As we notice that there is a lot of appending to the array going on, we can make it a bit easier with the following:

```bash
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


It basically makes the state dictionary as saw previously, and also makes sure that any new message is appended to the messages array when we do the following: 
```bash
{"messages": [new_array_element]}
```


##### We also realize that our app is not capable of answering simple questions like "how are you?"

In [69]:
inputs = {"messages": ["how are you?"]}
app.invoke(inputs)

"I am an AI assistant, so I don't have feelings. In Istanbul, the current weather is clear with a temperature of 15.68°C, a wind speed of 6.17 m/s, and 82% humidity. There is no rain, heat index, or cloud cover at the moment."

This is because we always want to parse a city and then find the weather. 

We can make our agent smarter by saying only use the tool when needed, if not just respond back to the user. 

The way we can do this LangGraph is:
1. binding a tool to the agent
2. adding a conditional edge to the agent with the option to either call the tool or not
3. defining the criteria for the conditional edge as when to call the tool. We will define a function for this.


Let's start with the AgentState definition as mentioned a few cells above.

In [70]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

Binding tool with agent (LLM Model) is made easy in langchain

In [71]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun
from langchain_core.utils.function_calling import convert_to_openai_function

tools = [OpenWeatherMapQueryRun()]

model = ChatOpenAI(temperature=0, streaming=True)
functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

Our modified function_1 now becomes as below. The reason is, we are passing the human message as state and appending response to the state. Also, our agent now has a tool bound to it, that it can use.

In [72]:
def function_1(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

For function 2, we want it to setup a tool and call it. It's made easy to invoke a tool in LangChain by using ToolInvocation and executing it with ToolExecuter. Then we respond back as a FunctionMessage so that our agent (node 1) knows that the tool was used and a response from tool is available.

In [73]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

def function_2(state):
    messages = state['messages']
    last_message = messages[-1] # this has the query we need to send to the tool provided by the agent

    parsed_tool_input = json.loads(last_message.additional_kwargs["function_call"]["arguments"])

    # We construct an ToolInvocation from the function_call and pass in the tool name and the expected str input for OpenWeatherMap tool
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=parsed_tool_input['__arg1'],
    )
    
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)

    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)

    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

Finally, we define a function for the conditional edge, to help us figure out which direction to go (tool or user response)

We can benefit from the agent (LLM) response in LangChain, which has additional_kwargs to make a function_call with the name of the tool.

So our logic is, if function_call available in the additional_kwargs, then call tool if not then end the discussion and respond back to the user

In [74]:
def where_to_go(state):
    messages = state['messages']
    last_message = messages[-1]
    
    if "function_call" in last_message.additional_kwargs:
        return "continue"
    else:
        return "end"

Now with all of the changes above, our LangGraph app is modified as below:

In [75]:
# from langgraph.graph import Graph, END

# workflow = Graph()

# Or you could import StateGraph and pass AgentState to it
from langgraph.graph import StateGraph, END
workflow = StateGraph(AgentState)

workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

# The conditional edge requires the following info below.
# First, we define the start node. We use `agent`.
# This means these are the edges taken after the `agent` node is called.
# Next, we pass in the function that will determine which node is called next, in our case where_to_go().

workflow.add_conditional_edges("agent", where_to_go,{   # Based on the return from where_to_go
                                                        # If return is "continue" then we call the tool node.
                                                        "continue": "tool",
                                                        # Otherwise we finish. END is a special node marking that the graph should finish.
                                                        "end": END
                                                    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that if `tool` is called, then it has to call the 'agent' next. 
workflow.add_edge('tool', 'agent')

# Basically, agent node has the option to call a tool node based on a condition, 
# whereas tool node must call the agent in all cases based on this setup.

workflow.set_entry_point("agent")


app = workflow.compile()

We also pass the first message using HumanMessage component available in langchain, makes it easy to differentiate from AIMessage, and FunctionMessage

In [76]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the temperature in oakland")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the temperature in oakland'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Oakland"}', 'name': 'open_weather_map'}}, response_metadata={'finish_reason': 'function_call'}, id='run-af33ad76-7c23-449f-9e60-e9b4b38342e6-0'),
  FunctionMessage(content='In Oakland, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 5.66 m/s, direction: 260°\nHumidity: 55%\nTemperature: \n  - Current: 29.38°C\n  - High: 31.1°C\n  - Low: 26.45°C\n  - Feels like: 30.88°C\nRain: {}\nHeat index: None\nCloud cover: 75%', name='open_weather_map'),
  AIMessage(content='The current temperature in Oakland is 29.38°C.', response_metadata={'finish_reason': 'stop'}, id='run-78c2a7f3-6a51-431d-afc0-a2959398a68f-0')]}

In [77]:
inputs = {"messages": [HumanMessage(content="what is the temperature in oakland")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Oakland"}', 'name': 'open_weather_map'}}, response_metadata={'finish_reason': 'function_call'}, id='run-e5f5b90e-aa55-48f8-813a-964ddb7dd9bf-0')]}

---

Output from node 'tool':
---
{'messages': [FunctionMessage(content='In Oakland, the current weather is as follows:\nDetailed status: broken clouds\nWind speed: 5.66 m/s, direction: 260°\nHumidity: 55%\nTemperature: \n  - Current: 29.38°C\n  - High: 31.1°C\n  - Low: 26.45°C\n  - Feels like: 30.88°C\nRain: {}\nHeat index: None\nCloud cover: 75%', name='open_weather_map')]}

---

Output from node 'agent':
---
{'messages': [AIMessage(content='The current temperature in Oakland is 29.38°C.', response_metadata={'finish_reason': 'stop'}, id='run-2376df48-53c3-41dd-b4d5-6b4195d5ee9e-0')]}

---



Hopefully, that gives you a good understanding of how we built a LangGraph app and why we used different LC components.